# 共通処理

In [ ]:
import os
import sys

import h5py

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:
import json

try:
    with open('setting.json', 'r') as file:
        setting = json.load(file)
except FileNotFoundError:
    print("先にset_run.ipynbで解析runを設定してください")

hdf5_path = setting['hdf5_path']
hdf5_path


In [ ]:
from model.File.HDF5 import HDF5

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# hdf5内のlayer pathを取得する
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)

In [ ]:
def return_hdf5_data(hdf5_path: str, *, query: str, shape=None):
    with h5py.File(hdf5_path, 'r') as f_read:
        hdf = HDF5(f=f_read)
        
        to_data = hdf.search_data_path(query=query)
        
        if type(to_data) is str:
            return hdf.return_data(data_path=to_data, shape=shape)
        elif type(to_data) is list:
            print("複数のlayer pathが検索されています。")
        else:
            print("layer pathが見つかりませんでした。")

# ここから

In [ ]:
OD = return_hdf5_data(hdf5_path, query='OD')

In [ ]:
OD